# Coseismic modelling with Pyrocko using ARIA products

**Author**: David Bekaert - Jet Propulsion Laboratory, 
            Marius Isken - Geoforschungszentrum Potsdam 

This notebook demonstrates how the Sentinel 1 ARIA Geocoded UNWrapped interferogram (**GUNW**) products can be used for geophysical modelling using the **Pyrocko** package. We will demonstrate how to:
- Download ARIA standard products covering the Ridgecrest Earthquake
- Extract the data and meta layers needed for Pyrocko
- Perform inverse modeling to solve fault slip using Pyrocko

A detailed overview of the ARIA GUNW product with respect to processing, formatting, sampling, and data layers can be found on the [ARIA website](https://aria.jpl.nasa.gov/node/97). 

**Pyrocko** is a geophysical modelling package for geodetic and sesimological data. It includes various downsampling methods and covariance estimation for InSAR data, and supports joint inversion modelling of InSAR and GNSS observations in a Bayesian framework. A full description can be found on the [Pyrocko website](https://pyrocko.org). 


<div class="alert alert-warning">
<b>Potential errors:</b>
Both Pyrocko and ARIA-tools needs to be installed to run this notebook.
</div>

<div class="alert alert-danger">
<b>Download</b>:     
    
- The ***jupyter notebook* does not allow for interactive entering of your user-name and password, use the *jupyter terminal* instead** with the same command for interactive use.
</div>

## Overview of the dataset

In this notebook we will invert for fault slip as a result from the ridgecrest earthquake. 


In [ ]:
!ariaDownload.py

## 1. Example of ariaDownload output options

The user has the option to **download**, **count**, and generate a **kmz** corresponding to return of their search criteria. Below we provide example for each of these over Track 4.

### Count the number of products

To get a count of the number of products, without downloading data, provide the **`--output`** option with the **`count`** argument. To get information on the exact product filenames also include the verbose option **`-v`**.

In [ ]:
!ariaDownload.py --track 4 --output count

### Generate KMZ of products

To generate a Google Earth kmz, without downloading data, provide the **`--output`** option with the **`kmz`** argument. In case you also want to get information on the exact product filenames you can enbale the verbose option by adding **`-v`**. The result is shown in **Fig. 1**.

In [ ]:
!ariaDownload.py --track 4 --output kmz

<img src="./support_docs/track_004.png" alt="track" width="700">
<center><b>Fig. 1</b> Image of products available along Track 4. </center>

### Download products

To download the products, simply omit the **`--output`** option or specify it with the **`download`** argument. Examples for downloading all data on track 4 include:

`ariaDownload.py -t 004`

`ariaDownload.py -t 004 -o download`


By default, the products will be downloaded into the *./products* folder. You can specify a different location using the **`-w`** option, such as:

`ariaDownload.py -t 004 -w /insarHome`

<div class="alert alert-warning">
<b>Potential download failure:</b>
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add “ARIA Product Search” to their URS approved applications
</div>

## 2. Spatial-temporal subsetting

The **`ariaDownload.py`** program has a number of options availble for subsetting the search of products in the spatial and temporal domain, including **bounding box**, **start/end date**, **aquisition geometry**, **temporal baseline**, or **interferogram** options. Below, using the count option for brevity, we demonstrate each of them.

### Subsetting by Bounding Box

Bounding box can either be specified as **SNWE** coordinates (a string with quotation) or by providing a **shapefile** (GeoJSON or ESRI shapefiles) to the **`--bbox`** option. 

An example based on subsetting using **SNWE** coordinates and the corresponding kmz is shown in **Fig. 2**:

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count

<img src="./support_docs/crop_hr.png" alt="crop" width="700">

<center><b>Fig. 2</b> Image of products available along within bounding box 36.75S, 37.225N, -76.655W, -75.928E</center>

An example based on subsetting using a shape file:

In [ ]:
!ariaDownload.py --track 4 --bbox './support_docs/HR_North.shp' -o count

<img src="./support_docs/crop_shp.png" alt="crop" width="700">

<center><b>Fig. 3</b> Image of products available along within shapefile (blue shading). </center>

### Subsetting by start/end date

Subsetting in time can be done by specifying a range within which products need to fall. This is controlled using the **`--start YYYYMMDD` ** and **`--end YYYYMMDD`** options. By default the complete observational record is considered.

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --start 20190101 

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count -s 20190101  --end 20190401 --verbose

Note the use of the **`--verbose`** option to show the products found.

### Subsetting based on product temporal baseline

The **`--daysless`** and **`--daysmore`** options allow us to subset based on the temporal baseline, i.e. the time length (in days) between the reference and secondary SAR acquisitions of the interferogram. Note that `--daysless 12` and `--daysmore 12` specifies only pairs with temporal baselines smaller and larger than 12 (so not exactly separated by 12) are downloaded, respectively.

To select only interferograms products with temporal baselines shorter than 13 days:

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysless 13 -v

Select only pairs with temporal baselines, greater than a year:

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count --daysmore 364

Select only pairs with moderate temporal baselines between two and six months:

In [ ]:
!ariaDownload.py --track 4  --bbox "36.75 37.225 -76.655 -75.928"  -o count -m 59 -l 181 -v 

### Subsetting based on interferogram pair

To find a specific interferogram combination one can use the **`--ifg YYYYMMDD_YYYYMMDD`** option.

In [ ]:
!ariaDownload.py -b "36.75 37.225 -76.655 -75.928"  -o count --ifg "20161018_20160702" -v 

In [2]:
!wget https://aria-products.jpl.nasa.gov/search/dataset/grq_v2.0.2_s1-gunw-released/S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2/S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2.nc

# Descending
!wget https://aria-products.jpl.nasa.gov/search/dataset/grq_v2.0.2_s1-gunw-released/S1-GUNW-D-R-071-tops-20190728_20190622-135213-36450N_34472N-PP-b4b2-v2_0_2/S1-GUNW-D-R-071-tops-20190728_20190622-135213-36450N_34472N-PP-b4b2-v2_0_2.nc
    
    




--2020-02-19 21:09:50--  https://aria-products.jpl.nasa.gov/search/dataset/grq_v2.0.2_s1-gunw-released/S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2/S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2.nc
Resolving aria-products.jpl.nasa.gov (aria-products.jpl.nasa.gov)... 128.149.112.114
Connecting to aria-products.jpl.nasa.gov (aria-products.jpl.nasa.gov)|128.149.112.114|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: http://aria-ops-dataset-bucket.s3-website-us-west-2.amazonaws.com/datasets/interferogram/v2.0.2/2019/07/10/S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2/S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2.nc [following]
--2020-02-19 21:09:50--  http://aria-ops-dataset-bucket.s3-website-us-west-2.amazonaws.com/datasets/interferogram/v2.0.2/2019/07/10/S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2/S1-GUNW-A-R-064

connCompMapping [[ 0.  0.  0.  0.]
 [ 1.  1.  0.  0.]
 [ 2.  2.  0.  0.]
 [ 3.  3.  0.  0.]
 [ 4.  4.  0.  0.]
 [ 5.  5.  0.  0.]
 [ 6.  6.  0.  0.]
 [ 7.  7.  0.  0.]
 [ 8.  8.  0.  0.]
 [ 9.  9.  0.  0.]
 [10. 10.  0.  0.]
 [11. 11.  0.  0.]
 [12. 12.  0.  0.]
 [13. 13.  0.  0.]
 [14. 14.  0.  0.]
 [15. 15.  0.  0.]
 [16. 16.  0.  0.]
 [17. 17.  0.  0.]
 [18. 18.  0.  0.]
 [19. 19.  0.  0.]
 [20. 20.  0.  0.]]
connIDMapping [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. -1.]
intMapping [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
intShift [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
ERROR 1: /Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-clone/JupyterDocs/ariaPyrocko/IntermediateFiles_a2jdpuvt/unw/Product_0_intShift.tif, band 1: Failed to compute min/max, no valid pixels found in sampling.
Generating Connecte

In [9]:
# Extract into ascending/
!ariaExtract.py -w ascending -f S1-GUNW-A-R-064-tops-20190710_20180703-015013-36885N_35006N-PP-9955-v2_0_2.nc -d download -l unwrappedPhase,incidenceAngle,azimuthAngle,lookAngle -sm overlap

# Extract into descending/
!ariaExtract.py -w descending -f S1-GUNW-D-R-071-tops-20190728_20190622-135213-36450N_34472N-PP-b4b2-v2_0_2.nc  -d download -l unwrappedPhase,incidenceAngle,azimuthAngle,lookAngle -sm overlap



***Extract Product Function:***
All (1) GUNW products meet spatial bbox criteria.
Group GUNW products into spatiotemporally continuous interferograms.
All (1) interferograms are spatially continuous.
Thread count specified for gdal multiprocessing = 2
Downloaded 3 arc-sec SRTM DEM here: ascending/DEM/SRTM_3arcsec.dem
Generating: unwrappedPhase - [==================================================] 20190710_20180703 
Generating: incidenceAngle - [==================================================] 20190710_20180703 
Generating: azimuthAngle - [==================================================] 20190710_20180703 
Generating: lookAngle - [==================================================] 20190710_20180703 
***Extract Product Function:***
All (1) GUNW products meet spatial bbox criteria.
Group GUNW products into spatiotemporally continuous interferograms.
All (1) interferograms are spatially continuous.
Thread count specified for gdal multiprocessing = 2
Downloaded 3 arc-sec SRTM DEM he

In [10]:
!pwd

/Users/dbekaert/Software/ARIA_tools/ARIA-tools-docs-clone/JupyterDocs/ariaPyrocko


In [6]:

# now load the data
###scene.load
###scene.import_data
!conda init bash
!source activate ARIA-tools_latest

!conda activate kite
!spool --load ascending

no change     /Users/dbekaert/Software/python/miniconda37/condabin/conda
no change     /Users/dbekaert/Software/python/miniconda37/bin/conda
no change     /Users/dbekaert/Software/python/miniconda37/bin/conda-env
no change     /Users/dbekaert/Software/python/miniconda37/bin/activate
no change     /Users/dbekaert/Software/python/miniconda37/bin/deactivate
no change     /Users/dbekaert/Software/python/miniconda37/etc/profile.d/conda.sh
no change     /Users/dbekaert/Software/python/miniconda37/etc/fish/conf.d/conda.fish
no change     /Users/dbekaert/Software/python/miniconda37/shell/condabin/Conda.psm1
no change     /Users/dbekaert/Software/python/miniconda37/shell/condabin/conda-hook.ps1
no change     /Users/dbekaert/Software/python/miniconda37/lib/python3.7/site-packages/xonsh/conda.xsh
no change     /Users/dbekaert/Software/python/miniconda37/etc/profile.d/conda.csh
no change     /Users/dbekaert/.bash_profile
No action taken.

CommandNotFoundError: Your shell has not been properly conf

In [ ]:
from kite import Scene

# Import Matlab container to kite
scene = Scene.import_file('ascending')
scene.spool()  # start the GUI for data inspection and Quadtree parametrisation
